# 🧹 Preprocessing Template
Notebook ini digunakan untuk preprocessing data sebelum pemodelan.

## Langkah yang disarankan:
- Load dataset
- Cek dan tangani missing value
- Encode variabel kategorikal
- Normalisasi atau standarisasi
- Simpan dataset bersih ke file baru (opsional)

In [6]:
# Sel 1: Import library dan muat data mentah
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import sys

# Menambahkan folder root ke path
sys.path.append(os.path.abspath(os.path.join('..')))
from src import config

# Memuat data mentah
df_raw = pd.read_csv(config.RAW_DATA_PATH)
df_processed = df_raw.copy() # Buat salinan untuk diproses

print("Data mentah berhasil dimuat. Jumlah baris:", len(df_processed))

Data mentah berhasil dimuat. Jumlah baris: 106674


In [7]:
# Sel 2: Penanganan Nilai Kosong (Missing Values)
# --- Gunakan sel Markdown di atas ini dengan judul: "1. Penanganan Nilai Kosong" ---

# Kolom-kolom penting yang akan dicek
cols_to_check_na = ['la', 'ld', 'nf', 'ent', 'buggy', 'aexp', 'arexp', 'asexp', 'nd', 'ns']

print(f"Jumlah baris sebelum dropna: {len(df_processed):,}")
df_processed.dropna(subset=cols_to_check_na, inplace=True)
print(f"Jumlah baris sesudah dropna: {len(df_processed):,}")

Jumlah baris sebelum dropna: 106,674
Jumlah baris sesudah dropna: 106,674


In [8]:
# Sel 3: Rekayasa Fitur (Feature Engineering)
# --- Gunakan sel Markdown di atas ini dengan judul: "2. Rekayasa Fitur" ---

# Mengubah tipe data tanggal dan menghapus baris yang gagal diubah
df_processed['author_date'] = pd.to_datetime(df_processed['author_date'], errors='coerce')
df_processed.dropna(subset=['author_date'], inplace=True)

# Membuat fitur baru 'code_churn'
df_processed['code_churn'] = df_processed['la'] + df_processed['ld']

# Mengubah fitur 'fix' menjadi integer (0 atau 1)
df_processed['fix_int'] = df_processed['fix'].astype(int)

# Mengubah target 'buggy' menjadi boolean (True atau False)
df_processed['buggy'] = df_processed['buggy'].astype(bool)


print("Fitur baru ('code_churn', 'fix_int') berhasil dibuat.")
df_processed[['la', 'ld', 'code_churn', 'fix', 'fix_int', 'buggy']].head()

Fitur baru ('code_churn', 'fix_int') berhasil dibuat.


,la,ld,code_churn,fix,fix_int,buggy
0,372,23,395,False,0,False
1,2,2,4,False,0,False
2,41,26,67,False,0,False
3,8,6,14,False,0,False
4,70,4,74,False,0,False


In [9]:
# Sel 4: Definisikan Fitur (X) dan Target (y)
# --- Gunakan sel Markdown di atas ini dengan judul: "3. Pemisahan Fitur dan Target" ---
X = df_processed[config.FEATURES]
y = df_processed[config.TARGET]

print("Fitur (X) yang digunakan:")
print(X.columns.to_list())
print("\nBentuk X:", X.shape)
print("Bentuk y:", y.shape)

Fitur (X) yang digunakan:
['la', 'ld', 'nf', 'ent', 'code_churn', 'nd', 'ns', 'aexp', 'arexp', 'asexp', 'fix_int']

Bentuk X: (106674, 11)
Bentuk y: (106674,)


In [10]:
# Sel 5: Pisahkan data menjadi set training dan testing
# --- Gunakan sel Markdown di atas ini dengan judul: "4. Pemisahan Data (Train-Test Split)" ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=config.TEST_SIZE, 
    random_state=config.RANDOM_STATE, 
    stratify=y
)

print(f"Data training (X_train): {X_train.shape}")
print(f"Data testing (X_test):  {X_test.shape}")

Data training (X_train): (85339, 11)
Data testing (X_test):  (21335, 11)


In [11]:
# Sel 6: Normalisasi/Standardisasi Data
# --- Gunakan sel Markdown di atas ini dengan judul: "5. Normalisasi Fitur" ---
scaler = StandardScaler()

# Fit scaler HANYA pada data training untuk mencegah data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data berhasil di-scale.")

# Tampilkan perbandingan
print("\nContoh data SEBELUM di-scale (5 baris pertama X_train):")
print(X_train.head().to_string())
print("\nContoh data SESUDAH di-scale (5 baris pertama X_train_scaled):")
print(pd.DataFrame(X_train_scaled, columns=config.FEATURES).head().to_string())

Data berhasil di-scale.

Contoh data SEBELUM di-scale (5 baris pertama X_train):
        la   ld  nf       ent  code_churn  nd  ns  aexp  arexp      asexp  fix_int
74156   12   11   1  0.000000          23   1   1     3    3.0   3.000000        0
63151  146    2   2  0.179256         148   1   1     0    0.0   0.000000        0
34198  190   10   3  1.329889         200   1   1     0    0.0   0.000000        1
1300   184    0   1  0.000000         184   1   1    87   87.0  81.000000        0
72629    7  204   3  0.627258         211   3   2     8    8.0   1.027344        0

Contoh data SESUDAH di-scale (5 baris pertama X_train_scaled):
         la        ld        nf       ent  code_churn        nd        ns      aexp     arexp     asexp   fix_int
0 -0.333217 -0.210127 -0.529512 -1.050416   -0.351693 -0.569537 -0.437003 -0.451470 -0.597015 -0.402441 -0.335436
1 -0.079254 -0.239252 -0.430212 -0.905997   -0.169089 -0.569537 -0.437003 -0.452580 -0.599987 -0.405474 -0.335436
2  0.004137 -0.

In [12]:
# Sel 7: Simpan Data yang Sudah Diproses ke File CSV
# --- Gunakan sel Markdown di atas ini dengan judul: "6. Menyimpan Hasil Preprocessing" ---

# Mengubah array numpy kembali ke DataFrame agar mudah dibaca
X_train_df = pd.DataFrame(X_train_scaled, columns=config.FEATURES)
X_test_df = pd.DataFrame(X_test_scaled, columns=config.FEATURES)

# Menentukan path untuk setiap file
path_x_train = os.path.join(config.PROCESSED_DATA_PATH, "X_train_scaled.csv")
path_x_test = os.path.join(config.PROCESSED_DATA_PATH, "X_test_scaled.csv")
path_y_train = os.path.join(config.PROCESSED_DATA_PATH, "y_train.csv")
path_y_test = os.path.join(config.PROCESSED_DATA_PATH, "y_test.csv")

# Menyimpan file
X_train_df.to_csv(path_x_train, index=False)
X_test_df.to_csv(path_x_test, index=False)
y_train.to_csv(path_y_train, index=False)
y_test.to_csv(path_y_test, index=False)

print("Semua data yang telah diproses berhasil disimpan di folder 'data/processed/':")
print(f"- {os.path.basename(path_x_train)}")
print(f"- {os.path.basename(path_x_test)}")
print(f"- {os.path.basename(path_y_train)}")
print(f"- {os.path.basename(path_y_test)}")

Semua data yang telah diproses berhasil disimpan di folder 'data/processed/':
- X_train_scaled.csv
- X_test_scaled.csv
- y_train.csv
- y_test.csv
